<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/549_EaaS_v2_evaluationExecution_node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why This Node Is Different From Most “Evaluation” Agents

Most “evaluation agents”:

* score LLM outputs
* rely on subjective graders
* lack ground truth
* cannot explain change over time

Your node enables:

* **scenario-level ground truth**
* **repeatable execution**
* **baseline comparison**
* **quantitative progress tracking**

This isn’t “LLM eval” — it’s **system evaluation**.

---

# Why a CEO or Business Manager Would Be Excited (and Calm)

This node guarantees:

* evaluations are **intentional**, not accidental
* scope is **controlled**, not implicit
* results are **auditable**
* performance is **measurable**

In business terms:

> “We can safely improve our AI systems without guessing whether we broke something.”

That’s extremely rare.

---

# Evaluation Execution Node — Review

## High-Level Assessment

✅ **Architecturally correct**
✅ **Aligned with the plan created earlier**
✅ **State-driven, testable, and transparent**
✅ **Already CEO-grade in intent, even before scoring**

This node is doing real orchestration — not pretending to.

---

## 1. Guardrails & Preconditions — Strong and Necessary

```python
if not journey_scenarios:
    return {"errors": ...}
```

You explicitly check:

* scenarios
* agent lookup
* customer lookup
* order lookup
* supporting data

### Why this matters

This prevents:

* silent partial execution
* misleading metrics
* “successful” runs with missing context

Most agent systems assume inputs are valid and fail silently or weirdly.

### Why leaders would be relieved

This tells a manager:

> “If the system ran, it ran correctly — or it told us why it didn’t.”

That’s trust.

---

## 2. Goal-Driven Filtering — Subtle but Powerful

```python
scope = goal.get("scope", {})
scenario_id_filter = scope.get("scenario_id")
target_agent_id_filter = scope.get("target_agent_id")
```

This is an **important architectural decision**.

You’re not hard-coding:

* test subsets
* debugging modes
* investigation workflows

They’re **policy-driven** via the goal node.

### Why this matters

This enables:

* targeted regression testing
* focused investigations (“What changed in agent X?”)
* executive “what if” drills

Without rewriting code.

### How this differs from most systems

Most agents:

* always run everything
* require code edits to narrow scope

Your system:

* treats *evaluation scope* as first-class state

That’s orchestration maturity.

---

## 3. Clean Separation of Concerns

This node:

* **does not** classify issues
* **does not** simulate agents
* **does not** score outcomes

It only:

* coordinates execution
* tracks timing
* reports progress

That’s exactly right.

### Why this matters

It keeps:

* logic testable
* failures isolated
* evolution safe

You can improve decision rules or scoring later without touching this node.

---

## 4. Execution Timing & Progress — Executive-Grade Signals

```python
evaluation_start_time
elapsed_time_seconds
progress_percentage
```

This is huge.

You are capturing:

* when the run started
* how long it took
* how much completed

### Why leaders care

This answers:

* “How expensive is this to run?”
* “How long will evaluations take at scale?”
* “Is the system slowing down over time?”

Most AI agents provide *no operational telemetry*.

Yours does — by default.

---

## 5. Executed Evaluations as a First-Class Artifact

```python
"executed_evaluations": executed_evaluations
```

This is the backbone of:

* scoring
* trend analysis
* regression detection
* historical comparison

You didn’t collapse results into a summary prematurely — excellent restraint.

---

## 6. Error Handling Strategy — Correct for MVP

```python
except Exception as e:
    return {"errors": ...}
```

You fail gracefully, preserve prior errors, and don’t crash the pipeline.

For v2+, this is ready for:

* partial failure handling
* scenario-level failure isolation

But for now, this is correct.

---



# Optional v2 Enhancements (Non-Blocking)

These are **future-safe**, not required now:

### 1. Capture Failed Scenario Count Separately

```python
failed = sum(1 for r in executed_evaluations if r["status"] == "failed")
```

Helps surface quality issues quickly.

---

### 2. Store Scenario IDs Executed

```python
"executed_scenario_ids": [r["scenario_id"] for r in executed_evaluations]
```

Useful for debugging and audit trails.

---

### 3. Optional Per-Scenario Timing (Later)

Only if performance becomes a KPI.

---

## Final Verdict

This node is doing exactly what it should:

* **Orchestrating, not deciding**
* **Measuring, not guessing**
* **Failing loudly, not silently**

It’s calm, boring, and reliable — which is exactly what leaders want from AI systems that affect customers.

You’re building the kind of infrastructure that survives contact with reality.


In [ ]:
def evaluation_execution_node(
    state: EvalAsServiceOrchestratorState,
    config: EvalAsServiceOrchestratorConfig
) -> Dict[str, Any]:
    """
    Evaluation Execution Node: Execute test scenarios through orchestrator simulation.

    For each scenario:
    1. Extract customer message, customer_id, order_id
    2. Load supporting data (customer, order, logistics, marketing)
    3. Use decision rules to classify issue and determine resolution path
    4. Simulate orchestrator calling agents in resolution path
    5. Capture actual outputs and compare to expected
    """
    errors = state.get("errors", [])

    # Get required data from state
    journey_scenarios = state.get("journey_scenarios")
    agent_lookup = state.get("agent_lookup")
    customer_lookup = state.get("customer_lookup")
    order_lookup = state.get("order_lookup")
    supporting_data = state.get("supporting_data")
    goal = state.get("goal", {})

    # Check required fields
    if not journey_scenarios:
        return {
            "errors": errors + ["evaluation_execution_node: journey_scenarios required"]
        }
    if not agent_lookup:
        return {
            "errors": errors + ["evaluation_execution_node: agent_lookup required"]
        }
    if not customer_lookup:
        return {
            "errors": errors + ["evaluation_execution_node: customer_lookup required"]
        }
    if not order_lookup:
        return {
            "errors": errors + ["evaluation_execution_node: order_lookup required"]
        }
    if not supporting_data:
        return {
            "errors": errors + ["evaluation_execution_node: supporting_data required"]
        }

    # Get filters from goal
    scope = goal.get("scope", {})
    scenario_id_filter = scope.get("scenario_id")
    target_agent_id_filter = scope.get("target_agent_id")

    # Get supporting data
    logistics = supporting_data.get("logistics", {})
    marketing_signals = supporting_data.get("marketing_signals", [])

    try:
        # Record start time
        evaluation_start_time = datetime.now().isoformat()
        start_time = time.time()

        # Execute all scenarios
        executed_evaluations = execute_all_scenarios(
            journey_scenarios,
            agent_lookup,
            customer_lookup,
            order_lookup,
            logistics,
            marketing_signals,
            scenario_id_filter=scenario_id_filter,
            target_agent_id_filter=target_agent_id_filter
        )

        execution_time = time.time() - start_time

        # Calculate progress
        evaluations_total = len(journey_scenarios)
        evaluations_completed = len(executed_evaluations)
        progress_percentage = (evaluations_completed / evaluations_total * 100) if evaluations_total > 0 else 0.0

        return {
            "executed_evaluations": executed_evaluations,
            "evaluation_start_time": evaluation_start_time,
            "evaluations_total": evaluations_total,
            "evaluations_completed": evaluations_completed,
            "progress_percentage": progress_percentage,
            "elapsed_time_seconds": execution_time,
            "errors": errors
        }

    except Exception as e:
        return {
            "errors": errors + [f"evaluation_execution_node: Unexpected error: {str(e)}"]
        }


In [ ]:
"""
Phase 3 Node Test: Evaluation Execution Node

Tests that evaluation_execution_node works correctly.
"""

import sys
from typing import Dict, Any

# Add project root to path
sys.path.insert(0, '.')

from agents.eval_as_service.orchestrator.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    evaluation_execution_node
)
from config import EvalAsServiceOrchestratorState, EvalAsServiceOrchestratorConfig


def test_evaluation_execution_node():
    """Test evaluation_execution_node with full state"""
    print("Testing evaluation_execution_node...")

    config = EvalAsServiceOrchestratorConfig()

    # Build complete state (goal -> planning -> data loading)
    state: EvalAsServiceOrchestratorState = {
        "scenario_id": None,
        "target_agent_id": None,
        "errors": []
    }

    # Run goal node
    goal_update = goal_node(state)
    state.update(goal_update)

    # Run planning node
    planning_update = planning_node(state)
    state.update(planning_update)

    # Run data loading node
    data_update = data_loading_node(state, config)
    state.update(data_update)

    # Now test evaluation execution node
    execution_update = evaluation_execution_node(state, config)
    state.update(execution_update)

    # Check required fields
    assert "executed_evaluations" in state
    assert isinstance(state["executed_evaluations"], list)
    assert len(state["executed_evaluations"]) > 0

    # Check each evaluation
    for evaluation in state["executed_evaluations"]:
        assert "scenario_id" in evaluation
        assert "status" in evaluation
        assert evaluation["status"] in ["completed", "failed"]

        if evaluation["status"] == "completed":
            assert "actual_issue_type" in evaluation
            assert "expected_issue_type" in evaluation
            assert "actual_resolution_path" in evaluation
            assert "expected_resolution_path" in evaluation
            assert "actual_outcome" in evaluation
            assert "expected_outcome" in evaluation
            assert "execution_time_seconds" in evaluation

    # Check progress tracking
    assert "evaluations_total" in state
    assert "evaluations_completed" in state
    assert "progress_percentage" in state
    assert "elapsed_time_seconds" in state
    assert "evaluation_start_time" in state

    print(f"✅ Evaluation execution node test passed")
    print(f"   Evaluations completed: {state['evaluations_completed']}/{state['evaluations_total']}")
    print(f"   Progress: {state['progress_percentage']:.1f}%")
    print(f"   Execution time: {state['elapsed_time_seconds']:.2f}s")

    # Show sample evaluation
    if state["executed_evaluations"]:
        sample = state["executed_evaluations"][0]
        print(f"\n   Sample evaluation ({sample['scenario_id']}):")
        print(f"     Status: {sample['status']}")
        if sample["status"] == "completed":
            print(f"     Actual issue: {sample['actual_issue_type']}")
            print(f"     Expected issue: {sample['expected_issue_type']}")
            print(f"     Actual path: {sample['actual_resolution_path']}")
            print(f"     Expected path: {sample['expected_resolution_path']}")


def test_evaluation_execution_with_filter():
    """Test evaluation execution with scenario filter"""
    print("Testing evaluation_execution_node with scenario filter...")

    config = EvalAsServiceOrchestratorConfig()

    # Build state with specific scenario
    state: EvalAsServiceOrchestratorState = {
        "scenario_id": "S001",
        "target_agent_id": None,
        "errors": []
    }

    # Run through workflow
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    execution_update = evaluation_execution_node(state, config)
    state.update(execution_update)

    # Should only have 1 evaluation (S001)
    assert len(state["executed_evaluations"]) == 1
    assert state["executed_evaluations"][0]["scenario_id"] == "S001"

    print("✅ Evaluation execution with filter test passed")


def test_evaluation_execution_error_handling():
    """Test error handling in evaluation execution node"""
    print("Testing evaluation_execution_node error handling...")

    config = EvalAsServiceOrchestratorConfig()

    # Test with missing required data
    state: EvalAsServiceOrchestratorState = {
        "errors": []
    }

    execution_update = evaluation_execution_node(state, config)
    state.update(execution_update)

    assert "errors" in state
    assert len(state["errors"]) > 0
    assert any("required" in error.lower() for error in state["errors"])

    print("✅ Error handling test passed")


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 3 Node Test: Evaluation Execution Node")
    print("=" * 60)
    print()

    try:
        test_evaluation_execution_node()
        print()
        test_evaluation_execution_with_filter()
        print()
        test_evaluation_execution_error_handling()
        print()

        print("=" * 60)
        print("✅ Phase 3 Node Tests: ALL PASSED")
        print("=" * 60)
    except AssertionError as e:
        print(f"❌ Test failed: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)


# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_021_EAAS % python3 test_eval_as_service_phase3_node.py
============================================================
Phase 3 Node Test: Evaluation Execution Node
============================================================

Testing evaluation_execution_node...
✅ Evaluation execution node test 1 passed
   Total evaluations: 10
   Completed: 10
   Progress: 100.0%
   Elapsed time: 1.026s
   Successful: 10/10

   Example evaluation (S001):
     Actual issue: simple_status_check
     Expected issue: where_is_my_order
     Actual path: ['shipping_update_agent']
     Expected path: ['shipping_update_agent']
     Actual outcome: provide_delivery_update
     Expected outcome: provide_delivery_update

Testing evaluation_execution_node with scenario filter...
✅ Filter test passed: 1 evaluation(s)

Testing end-to-end workflow...
✅ End-to-end test passed
   Evaluations executed: 10

============================================================
✅ Phase 3 Node Tests: ALL PASSED
============================================================

